In [1]:
from tensorflow.keras.layers import LSTM

class CustomLSTM(LSTM):
    def __init__(self, *args, **kwargs):
        kwargs.pop('time_major', None)  # Remove the 'time_major' argument if it exists
        super(CustomLSTM, self).__init__(*args, **kwargs)

from tensorflow.keras.models import load_model
from tensorflow.keras.utils import get_custom_objects

get_custom_objects().update({'CustomLSTM': CustomLSTM})

trained_model = load_model(r"C:\Users\pranali\Desktop\Throughput_prediction\4G-Models\Models\univariate_classifier.hdf5", custom_objects={'LSTM': CustomLSTM})

trained_model.summary()

C:\Users\pranali\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
C:\Users\pranali\anaconda3\Lib\site-packages\keras\src\optimizers\base_optimizer.py:33: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (CustomLSTM)               │ (None, 10, 160)        │       103,680 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 10, 160)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (CustomLSTM)             │ (None, 10, 128)        │       147,968 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 10, 128)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (CustomLSTM)             │ (None, 10, 80)         │        66,880 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 10, 80)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 800)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 48)             │        38,448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 48)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 56)             │         2,744 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 56)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 48)             │         2,736 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 48)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ OUT_univariate_classifier       │ (None, 3)              │           147 │
│ (Dense)                         │                        │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 362,605 (1.38 MB)

 Trainable params: 362,603 (1.38 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

In [3]:
#4g

import numpy as np
import tensorflow as tf


X_train = np.load(r"C:\Users\pranali\Desktop\Throughput_prediction\test_train_vis\Training\univariate_classifier_train_x.npy")  # Shape should be (num_samples, time_steps, num_features)
y_train = np.load(r"C:\Users\pranali\Desktop\Throughput_prediction\test_train_vis\Training\univariate_classifier_train_y.npy")  # Shape should be (num_samples, num_classes)
X_test = np.load(r"C:\Users\pranali\Desktop\Throughput_prediction\test_train_vis\Testing\univariate_classifier_test_x.npy")
y_test = np.load(r"C:\Users\pranali\Desktop\Throughput_prediction\test_train_vis\Testing\univariate_classifier_test_y.npy")

# Ensure reproducibility
np.random.seed(42)
tf.random.set_seed(42)

In [5]:
import tensorflow_model_optimization as tfmot
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import get_custom_objects
import tensorflow.keras.backend as K



#quantized model default

# Convert the model to a TensorFlow Lite model with post-training quantization
converter = tf.lite.TFLiteConverter.from_keras_model(trained_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]

# Enable Select TF ops to support operations not natively supported by TFLite
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS, 
                                       tf.lite.OpsSet.SELECT_TF_OPS]

# Disable lowering of tensor list operations, which might be causing the error
converter._experimental_lower_tensor_list_ops = False

# Enable resource variable support
converter.experimental_enable_resource_variables = True

# Convert the model
tflite_quant_model = converter.convert()

INFO:tensorflow:Assets written to: C:\Users\pranali\AppData\Local\Temp\tmprzo1df1k\assets


INFO:tensorflow:Assets written to: C:\Users\pranali\AppData\Local\Temp\tmprzo1df1k\assets


Saved artifact at 'C:\Users\pranali\AppData\Local\Temp\tmprzo1df1k'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 10, 1), dtype=tf.float32, name='lstm_input')
Output Type:
  TensorSpec(shape=(None, 3), dtype=tf.float32, name=None)
Captures:
  2451805175440: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2451773795280: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2451805172368: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2451805175248: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2451806423952: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2451806424144: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2451806422416: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2451806426064: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2451806425104: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2451806427792: TensorSpec(shape=(), dtype=tf.resource, name=None)
  24518064283

In [7]:
# Save the TFLite model
with open('4g_classifier_dynamic.tflite', 'wb') as f:
    f.write(tflite_quant_model)

In [9]:
# Load the quantized TFLite model
interpreter = tf.lite.Interpreter(model_path='4g_classifier_dynamic.tflite')
interpreter.allocate_tensors()

# Get details of input and output tensors
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

In [11]:
import time

# Initialize list to store predictions
predictions = []

# Measure inference time
start_time = time.time()
for i in range(X_test.shape[0]):
    # Prepare the input data (trim the extra dimension if necessary)
    input_data = X_test[i]  # Trim to match the expected shape (1, 10, 10)
    input_data = np.expand_dims(input_data, axis=0).astype(np.float32)
    
    # Set the input tensor
    interpreter.set_tensor(input_details[0]['index'], input_data)
    
    # Run inference
    interpreter.invoke()
    
    # Get the output tensor
    output_data = interpreter.get_tensor(output_details[0]['index'])
    
    # Append prediction to the list
    predictions.append(output_data)

end_time = time.time()
total_inference_time = end_time - start_time
avg_inference_time = total_inference_time / len(X_test)

print(f"Total Inference Time for quantized classifier 4g sample: {total_inference_time:.6f} seconds")
print(f"Avg Inference Time for quantized classifier 4g sample: {avg_inference_time:.6f} seconds")


Total Inference Time for quantized classifier 4g sample: 63.802561 seconds
Avg Inference Time for quantized classifier 4g sample: 0.002395 seconds


In [29]:
import os
# Get the size of the saved quantized TFLite model
model_size = os.path.getsize('4g_classifier_dynamic.tflite')
print(f"Quantized TFLite model size: {model_size / 1024:.2f} KB")  # Convert to KB

Quantized TFLite model size: 416.44 KB


In [33]:
import numpy as np
import tensorflow as tf
from sklearn.metrics import confusion_matrix, classification_report

# Load the quantized TFLite model
interpreter = tf.lite.Interpreter(model_content=tflite_quant_model)
interpreter.allocate_tensors()

# Get input and output details
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Function to run inference using the interpreter
def run_inference(interpreter, X_test):
    predictions = []
    for i in range(len(X_test)):
        # Preprocess the input as per your model's input requirements
        input_data = np.expand_dims(X_test[i], axis=0).astype(np.float32)  # Adjust datatype if needed
        interpreter.set_tensor(input_details[0]['index'], input_data)
        interpreter.invoke()
        output_data = interpreter.get_tensor(output_details[0]['index'])
        predictions.append(output_data)
    return np.array(predictions)

# Assume X_test and y_test are your test data and labels
# Run inference on the quantized model
predictions = run_inference(interpreter, X_test)

# Convert predictions to the correct shape if needed (e.g., flattening)
predictions = np.squeeze(predictions)

# For classification, get the predicted class and the true class
predicted_classes = np.argmax(predictions, axis=1)
true_classes = np.argmax(y_test, axis=1)

# Print the confusion matrix
conf_matrix = confusion_matrix(true_classes, predicted_classes)
print("Confusion Matrix:")
print(conf_matrix)

# Print the classification report
target_names = ["Low (0)", "Medium (1)", "High (2)"]
class_report = classification_report(true_classes, predicted_classes, target_names=target_names)
print("\nClassification Report:")
print(class_report)

# Optional: If you need to compute accuracy again
accuracy = np.sum(predicted_classes == true_classes) / len(true_classes)
print("Accuracy:", accuracy)


Confusion Matrix:
[[  561     0  3434]
 [  101     0  7890]
 [   67     0 14590]]

Classification Report:
              precision    recall  f1-score   support

     Low (0)       0.77      0.14      0.24      3995
  Medium (1)       0.00      0.00      0.00      7991
    High (2)       0.56      1.00      0.72     14657

    accuracy                           0.57     26643
   macro avg       0.44      0.38      0.32     26643
weighted avg       0.43      0.57      0.43     26643

Accuracy: 0.5686671921330181


C:\Users\pranali\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\pranali\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\pranali\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [43]:
import numpy as np
import tensorflow as tf
from sklearn.metrics import confusion_matrix, classification_report

# Load the quantized TFLite model
interpreter = tf.lite.Interpreter(model_content=tflite_quant_model)
interpreter.allocate_tensors()

# Get input and output details
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Function to run inference using the interpreter
def run_inference(interpreter, X_test):
    predictions = []
    for i in range(len(X_test)):
        # Preprocess the input as per your model's input requirements
        input_data = np.expand_dims(X_test[i], axis=0).astype(np.float32)  # Adjust datatype if needed
        interpreter.set_tensor(input_details[0]['index'], input_data)
        interpreter.invoke()
        output_data = interpreter.get_tensor(output_details[0]['index'])
        predictions.append(output_data)
    return np.array(predictions)

# Assume X_test and y_test are your test data and labels
# Run inference on the quantized model
predictions = run_inference(interpreter, X_test)

# Convert predictions to the correct shape if needed (e.g., flattening)
predictions = np.squeeze(predictions)

# For classification, get the predicted class and the true class
predicted_classes = np.argmax(predictions, axis=1)
true_classes = np.argmax(y_test, axis=1)

# Print the confusion matrix
conf_matrix = confusion_matrix(true_classes, predicted_classes)
print("Confusion Matrix:")
print(conf_matrix)

# Print the classification report
target_names = ["Low (0)", "Medium (1)", "High (2)"]
class_report = classification_report(true_classes, predicted_classes, target_names=target_names)
print("\nClassification Report:")
print(class_report)

# Optional: If you need to compute accuracy again
accuracy = np.sum(predicted_classes == true_classes) / len(true_classes)
print("Accuracy:", accuracy)

# Run inference on the quantized model
predictions = run_inference(interpreter, X_test)

# Convert predictions to the correct shape if needed (e.g., flattening)
predictions = np.squeeze(predictions)

# Compute loss (assuming Mean Squared Error for regression, adjust for classification)
mse = np.mean(np.square(predictions - y_test))
print("Mean Squared Error (MSE):", mse)


Confusion Matrix:
[[  561     0  3434]
 [  101     0  7890]
 [   67     0 14590]]

Classification Report:
              precision    recall  f1-score   support

     Low (0)       0.77      0.14      0.24      3995
  Medium (1)       0.00      0.00      0.00      7991
    High (2)       0.56      1.00      0.72     14657

    accuracy                           0.57     26643
   macro avg       0.44      0.38      0.32     26643
weighted avg       0.43      0.57      0.43     26643

Accuracy: 0.5686671921330181


C:\Users\pranali\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\pranali\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\pranali\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Mean Squared Error (MSE): 0.28719025782544955
